In [1]:
# TODO:
# Use NetKet to generate groundstate of simple spin model as function of parameter
# Use NetKet to either measure observables, or use other representation of the model (weights) as input

# Use 03_ notebook to implement the 'train on extremes and predict in the middle'

In [2]:
# import jax.numpy as jnp
# import netket as nk
# import flax
# import flax.linen as nn
# from netket.operator.spin import sigmax,sigmaz 

# N=20

# hi = nk.hilbert.Spin(s=1 / 2, N=N)

# Gamma=-1
# H = sum([Gamma*sigmax(hi,i) for i in range(N)])

# V=-1
# H += sum([V*sigmaz(hi,i)*sigmaz(hi,(i+1)%N) for i in range(N)])

# class FFN(nn.Module):
    
#     alpha : int = 1
            
#     @nn.compact
#     def __call__(self, x):

#         # the first layer is a dense layer
#         dense = nn.Dense(features=self.alpha*x.shape[-1])
        
#         y=dense(x)

#         # the non-linearity is a simple ReLu    
#         y=nn.relu(y)
        
#         # sum the output
#         return jnp.sum(y,axis=-1)
    
# model=FFN(alpha=1)

# sampler = nk.sampler.MetropolisLocal(hi)

# vstate = nk.vqs.MCState(sampler, model, n_samples=1000)

# optimizer = nk.optimizer.Sgd(learning_rate=0.1)

# gs = nk.driver.VMC(H, optimizer, variational_state=vstate,preconditioner=nk.optimizer.SR(diag_shift=0.1))

# log=nk.logging.RuntimeLog()
# gs.run(n_iter=300,out=log)

# ffn_energy=vstate.expect(H)

In [3]:
import jax.numpy as jnp
import netket as nk
import flax
import flax.linen as nn
from netket.operator.spin import sigmax,sigmaz 

N=20

hi = nk.hilbert.Spin(s=1 / 2, N=N)

In [4]:
import netket.nn as nknn

graph=nk.graph.Chain(length=N,pbc=True)

class SymmModel(nknn.Module):
    alpha: int

    @nknn.compact
    def __call__(self, x):
        
        x = nknn.DenseSymm(symmetries=graph.translation_group(),
                           features=self.alpha,
                           kernel_init=nk.nn.initializers.normal(stddev=0.01))(x)
        x = nn.relu(x)

        # sum the output
        return jnp.sum(x,axis=(-1,-2))

In [5]:
model=SymmModel(alpha=4)

In [6]:
sampler = nk.sampler.MetropolisLocal(hi)

In [7]:
def get_gs_sample(H, sampler, model, n_samples):
    
    vstate = nk.vqs.MCState(sampler, model, n_samples=1000)

    optimizer = nk.optimizer.Sgd(learning_rate=0.1)

    gs = nk.driver.VMC(H, optimizer, variational_state=vstate,preconditioner=nk.optimizer.SR(diag_shift=0.1))

    log=nk.logging.RuntimeLog()
    gs.run(n_iter=300,out=log)
    
    s = vstate.sample()
    s = s.reshape(-1, s.shape[2])[:n_samples,:]
    
    return s

In [8]:
all_data = {}

for g in jnp.arange(0.1,2,0.1):
    H = sum([-g*sigmax(hi,i) for i in range(N)])

    J=-1
    H += sum([J*sigmaz(hi,i)*sigmaz(hi,(i+1)%N) for i in range(N)])

    s=get_gs_sample(H, sampler, model, 100)

    all_data[str(g)] = s

100%|██████████| 300/300 [00:15<00:00, 19.94it/s, Energy=-40.68076 ± 0.00011 [σ²=0.00001, R̂=1.0009]]   


# Supervised learning

In [9]:
def get_training_data(all_data, Ts, Tc=2.7, train_fraction=0.8):
    # Lists to store the raw data
    raw_T = []
    raw_x = []
    raw_y = [] 
    
    for T in Ts:      
        raw_x.append(all_data['%.3f'%(T)])
        n = len(all_data['%.3f'%(T)])
        label = [1,0] if T < Tc else [0,1]
        raw_y.append(np.array([label] * n))
        raw_T.append(np.array([T]*n))
        
    raw_T = np.concatenate(raw_T)
    raw_x = np.concatenate(raw_x, axis=0)
    raw_y = np.concatenate(raw_y, axis=0)
    
    # Shuffle
    indices = np.random.permutation(len(raw_x))
    all_T = raw_T[indices]
    all_x = raw_x[indices]
    all_y = raw_y[indices]

    # Split into train and test sets
    train_split = int(train_fraction * len(all_x))
    train_T = jnp.array(all_T[:train_split])
    train_x = jnp.array(all_x[:train_split])
    train_y = jnp.array(all_y[:train_split])
    test_T = jnp.array(all_T[train_split:])
    test_x = jnp.array(all_x[train_split:])
    test_y = jnp.array(all_y[train_split:])
    
    return [raw_T, raw_x, raw_y], [train_T, train_x, train_y], [test_T, test_x, test_y]